In [2]:
import pandas as pd
import numpy as np
from extract import *

In [3]:
from clean import *

In [4]:
ADDRESS = "https://openpowerlifting.gitlab.io/opl-csv/files/openpowerlifting-latest.zip"

df = pd.read_csv(extract_dataset(ADDRESS))

/var/folders/tn/my7fmgyj3gx8811h_mwsw1mh0000gn/T/ipykernel_13791/987062614.py:3: DtypeWarning: Columns (33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extract_dataset(ADDRESS))


In [5]:
# Fix Lift data
"""Fill in null data for non-completed lifts for all attempts of all lifts"""
replace_lift_null_data(df)

"""Properly returns best lift for each respective lift"""
checks_best_lift(df)

"""Calculates total for all three lifts"""
calculate_total(df)

"""Returns boolean for new column: 'ValidComp' if non-zero total and qualified i.e. no disqualifications, not showing up, failing all lifts, etc."""
df = validate_comp(df)

/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ValidComp"] = df.apply(lambda row: True if (str.isdigit(row["Place"]) and row["TotalKg"] != 0) else False, axis=1)


In [6]:
# Fix Weight data
"""Removes rows where no WeightClass nor Bodyweight are available"""
df = drop_null_weights(df)

"""Fixes all entries where WeightClass was simply a '+', takes bodyweight and uses that as the weight class"""
fix_plus_weightclass(df)

"""Created IPF Weight Class, takes WeightClass entry and removes any plus signs, assigns them into bins according to IPF weight classes"""
df = redefine_weight_classes(df)

/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_female["IPFWeightClassKg"] = pd.cut(df_female["IPFWeightClassKg"], bins = F_bins, labels = F_labels)
/Users/wlodarczyk/PROJECTS/PowerliftingVisuals/assets/clean.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_male['IPFWeightClassKg'] = pd.cut(df_male['IPFWeightClassKg'], bins = M_bins, labels = M_labels)


In [7]:
# Fix testing, metric and federation data
"""If null values False, if Yes gives True"""
assume_untested(df)

"""Fill columns with no entry with placeholder"""
fill_unknown_values(df)

"""For lifters with no parent federation, assume federation they belong to is the parent federation"""
assume_federation(df)

"""All metrics with null values, taken to be zero"""
fill_lift_scores(df)

In [10]:
df.iloc[:,:10].info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2847986 entries, 0 to 2865620
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Name            2847986 non-null  object 
 1   Sex             2847986 non-null  object 
 2   Event           2847986 non-null  object 
 3   Equipment       2847986 non-null  object 
 4   Age             1810836 non-null  float64
 5   AgeClass        2062294 non-null  object 
 6   BirthYearClass  1879353 non-null  object 
 7   Division        2847986 non-null  object 
 8   BodyweightKg    2828730 non-null  float64
 9   WeightClassKg   2847986 non-null  object 
dtypes: float64(2), object(8)
memory usage: 239.0+ MB


In [9]:
df.iloc[4395]

Name                   Galyna Danylenko
Sex                                   F
Event                               SBD
Equipment                         Wraps
Age                                27.0
AgeClass                          24-34
BirthYearClass                    24-39
Division                   Amateur Open
BodyweightKg                       42.3
WeightClassKg                        44
Squat1Kg                          -95.0
Squat2Kg                          -95.0
Squat3Kg                           95.0
Squat4Kg                            0.0
Best3SquatKg                       95.0
Bench1Kg                           35.0
Bench2Kg                           37.5
Bench3Kg                           40.0
Bench4Kg                            0.0
Best3BenchKg                       40.0
Deadlift1Kg                        90.0
Deadlift2Kg                       -95.0
Deadlift3Kg                         0.0
Deadlift4Kg                         0.0
Best3DeadliftKg                    90.0
